# Vegetable and Fruits Price Analysis

This mini-project intends to focus on the historical data of fruit and vegetable prices in Nepal between 2013 and 2021. The main focus is to employ a standard exploratory analysis and additionally explore machine learning forecasting. This is intended as practice and such forecasting is useful in the aid of future pricing, price trends, seasonality, anomalies and potential pricing strategies that can benefit agricultural workers and buyers.

The agriculture industry is an inherently complex and dynamic environment wherein forecasting future prices, production etc, is important for optimal decision making and resource allocation in the interest of optimal growth.
The complexity in trying to predict the prices of fruit vegetables is a complex task due to the sensitivity of price to the likes of crop yield, climate, global markets and production.

In [ ]:
import numpy as np
import math
import os, pathlib

import pandas as pd
import seaborn as sns
sns.set()

In [10]:
dataframe = pd.read_csv("kalimati_tarkari_dataset.csv")

dataframe.head(15)


,SN,Commodity,Date,Unit,Minimum,Maximum,Average
0,0,Tomato Big(Nepali),2013-06-16,Kg,35.0,40.0,37.5
1,1,Tomato Small(Local),2013-06-16,Kg,26.0,32.0,29.0
2,2,Potato Red,2013-06-16,Kg,20.0,21.0,20.5
3,3,Potato White,2013-06-16,Kg,15.0,16.0,15.5
4,4,Onion Dry (Indian),2013-06-16,Kg,28.0,30.0,29.0
5,5,Carrot(Local),2013-06-16,Kg,30.0,35.0,32.5
6,6,Cabbage(Local),2013-06-16,Kg,6.0,10.0,8.0
7,7,Cauli Local,2013-06-16,Kg,30.0,35.0,32.5
8,8,Raddish Red,2013-06-16,Kg,35.0,40.0,37.5
9,9,Raddish White(Local),2013-06-16,Kg,25.0,30.0,27.5


In [8]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197161 entries, 0 to 197160
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   SN         197161 non-null  int64  
 1   Commodity  197161 non-null  object 
 2   Date       197161 non-null  object 
 3   Unit       197161 non-null  object 
 4   Minimum    197161 non-null  float64
 5   Maximum    197161 non-null  float64
 6   Average    197161 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 10.5+ MB


In [ ]:
dataframe.describe()

,SN,Minimum,Maximum,Average
count,197161.000000,197161.000000,197161.000000,197161.000000
mean,98580.000000,85.423938,94.161284,89.792611
std,56915.622549,77.058896,82.375858,79.619003
min,0.000000,1.000000,6.000000,5.000000
25%,49290.000000,40.000000,45.000000,42.500000
50%,98580.000000,60.000000,70.000000,65.000000
75%,147870.000000,100.000000,110.000000,105.000000
max,197160.000000,1800.000000,2000.000000,1900.000000
